In [1]:
from groq import Groq
from dotenv import load_dotenv
import re
from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.discovery import build
import textwrap
from gtts import gTTS
import serpapi
import os, requests, json
import tempfile
import webbrowser


load_dotenv()

# Initialize Groq client with API key from .env
client = Groq(
    api_key=os.getenv("GROQ_API_KEY") 
)

# Initialize YouTube API client
youtube = build('youtube', 'v3', developerKey=os.getenv("YOUTUBE_API_KEY"))



In [2]:
def get_search_result(query):
    """Perform web search to answer questions about current events, people, or general knowledge"""
    client_serpapi = serpapi.Client(api_key=os.getenv("SERP_API_KEY"))
    r = client_serpapi.search({
        "q": query,
        "engine": "google",
    })

    results = ''
    for res in r['organic_results']:
        results += res['snippet'] + '\n'

    return results

def search_youtube_video(query):
    """Search for YouTube videos based on the query"""
    # Use YouTube API to search for videos
    request = youtube.search().list(
        q=query,
        part="snippet",
        type="video",
        maxResults=5
    )
    response = request.execute()
    
    # Return the top result
    if response['items']:
        video_id = response['items'][0]['id']['videoId']
        video_title = response['items'][0]['snippet']['title']
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        return {
            "video_id": video_id,
            "title": video_title,
            "url": video_url
        }
    else:
        return None

def extract_youtube_transcript(video_id):
    """Extract transcript from a YouTube video"""
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = ' '.join([item['text'] for item in transcript_list])
        return transcript_text
    except Exception as e:
        print(f"Error extracting transcript: {str(e)}")
        return None

def get_video_details(video_id):
    """Get additional details about a YouTube video"""
    request = youtube.videos().list(
        part="snippet,statistics,contentDetails",
        id=video_id
    )
    response = request.execute()
    
    if response['items']:
        video_info = response['items'][0]
        return {
            "title": video_info['snippet']['title'],
            "description": video_info['snippet']['description'],
            "channel": video_info['snippet']['channelTitle'],
            "publish_date": video_info['snippet']['publishedAt'],
            "view_count": video_info['statistics'].get('viewCount', 'N/A'),
            "like_count": video_info['statistics'].get('likeCount', 'N/A'),
            "comment_count": video_info['statistics'].get('commentCount', 'N/A'),
            "duration": video_info['contentDetails']['duration']
        }
    else:
        return None

def summarize_video_content(transcript, video_details):
    """Generate a summary of the video content using the LLM"""
    # Create a prompt for the LLM to summarize the video
    prompt = f"""
    Video Title: {video_details['title']}
    Channel: {video_details['channel']}
    
    Create a comprehensive summary of this video based on the transcript below.
    Include:
    1. A brief but detailed summary of the main content
    2. Key themes or points discussed
    3. Important characters or people mentioned
    4. The overall tone and sentiment of the video
    5. Any significant plot twists or revelations (if applicable)
    
    Make the summary feel like the person has actually watched the video.
    
    Transcript:
    {transcript[:8000]}  # Limit transcript length to avoid token limits
    """
    
    # Call the LLM API to generate the summary
    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are an expert video content analyzer. Your task is to provide comprehensive summaries of videos based on their transcripts."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=2000
    )
    
    return response.choices[0].message.content

def generate_speech(text):
    """Generate speech from text using gTTS"""
    try:
        # Create a temporary file for the audio
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as temp_audio:
            tts = gTTS(text=text, lang='en')
            tts.save(temp_audio.name)
            return temp_audio.name
    except Exception as e:
        print(f"Error generating speech: {str(e)}")
        return None

def analyze_sentiment(transcript):
    """Analyze sentiment of the video content"""
    prompt = f"""
    Analyze the sentiment and emotional tone of the following video transcript.
    Provide:
    1. Overall sentiment (positive, negative, neutral, mixed)
    2. Key emotions expressed
    3. Any noticeable shifts in tone
    
    Transcript:
    {transcript[:5000]}  # Limit transcript length
    """
    
    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are an expert in sentiment analysis."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )
    
    return response.choices[0].message.content

def extract_video_id_from_url(url):
    """Extract video ID from YouTube URL"""
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None

def understand_youtube_video(input_query, generate_audio=False, analyze_sentiment_flag=False):
    """Main function to understand YouTube videos without watching them"""
    # Step 1: Determine if input is a URL or a search query
    if "youtube.com" in input_query or "youtu.be" in input_query:
        video_id = extract_video_id_from_url(input_query)
        if not video_id:
            return {"error": "Invalid YouTube URL"}
        
        video_details = get_video_details(video_id)
        if not video_details:
            return {"error": "Could not fetch video details"}
        
        video_url = f"https://www.youtube.com/watch?v={video_id}"
    else:
        # Search for the video
        search_result = search_youtube_video(input_query)
        if not search_result:
            return {"error": "No videos found matching your query"}
        
        video_id = search_result["video_id"]
        video_url = search_result["url"]
        video_details = get_video_details(video_id)
    
    # Step 2: Extract transcript
    transcript = extract_youtube_transcript(video_id)
    if not transcript:
        return {
            "error": "Could not extract transcript. This might be due to unavailable captions or restricted content.",
            "video_url": video_url,
            "video_details": video_details
        }
    
    # Step 3: Generate summary
    summary = summarize_video_content(transcript, video_details)
    
    # Step 4: Additional analysis (if requested)
    sentiment_analysis = None
    if analyze_sentiment_flag:
        sentiment_analysis = analyze_sentiment(transcript)
    
    # Step 5: Generate audio (if requested)
    audio_file = None
    if generate_audio:
        # Use a shorter version of the summary for audio
        audio_summary = textwrap.shorten(summary, width=2000, placeholder="...")
        audio_file = generate_speech(audio_summary)
    
    # Step 6: Prepare and return the results
    result = {
        "video_title": video_details["title"],
        "channel": video_details["channel"],
        "video_url": video_url,
        "publish_date": video_details["publish_date"],
        "view_count": video_details["view_count"],
        "summary": summary
    }
    
    if sentiment_analysis:
        result["sentiment_analysis"] = sentiment_analysis
    
    if audio_file:
        result["audio_file"] = audio_file
        # Optionally open the audio file
        webbrowser.open(audio_file)
    
    return result

# Example usage
if __name__ == "__main__":
    # Input can be a video title, search query, or YouTube URL
    input_query = "Love in Every Word (Odogwu Paranra) by Omoni Oboli"
    
    # Set to True to generate audio reading of the summary
    generate_audio = False
    
    # Set to True to include sentiment analysis
    analyze_sentiment_flag = True
    
    result = understand_youtube_video(input_query, generate_audio, analyze_sentiment_flag)
    
    # Print the result
    print(f"Video Title: {result['video_title']}")
    print(f"Channel: {result['channel']}")
    print(f"Video URL: {result['video_url']}")
    print(f"Published on: {result['publish_date']}")
    print(f"View Count: {result['view_count']}")
    print("\nSummary:")
    print(result['summary'])
    
    if 'sentiment_analysis' in result:
        print("\nSentiment Analysis:")
        print(result['sentiment_analysis'])
    
    if 'audio_file' in result:
        print(f"\nAudio summary saved to: {result['audio_file']}")

Video Title: When Odogwu Paranran is your husband! LOVE IN EVERY WORD Latest Nigerian Full Movie 2025
Channel: Omoni Oboli TV
Video URL: https://www.youtube.com/watch?v=hwx6dU0AigM
Published on: 2025-03-10T14:10:05Z
View Count: 323607

Summary:
Summary:
In this video, the main character, Odogwu Paranran's girlfriend, is having a heated conversation with an unnamed individual, likely a coworker, regarding a work-related campaign that she is in charge of. She sternly warns the person not to threaten or disrespect her again, or she will take legal action against the organization. She also asserts her authority in the campaign, stating that she will not continue unless she is explicitly given control over it. If not, she threatens to have her boyfriend, Odogwu Paranran, use his influence to make her the CEO of the organization and fire the individual and the directors.

Key Themes:

* Professional power dynamics
* Threats and intimidation
* Love and support in a relationship
* Workplace ab